<a href="https://colab.research.google.com/github/jyporse/YOLO/blob/main/Test_Pok%C3%A9mon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cmake
!pip install dlib

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import dlib
import time

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!nvidia-smi

In [ ]:
import cv2
import numpy as np
import time

min_confidence = 0.5
width = 800
height = 0
show_ratio = 1.0
path = "/content/drive/MyDrive/Kobot/Pokémon/"
file_name = "/content/1.jpg"
Weights = path + "weight/custom-train-yolo_1200.weights"
test_cfg = path +"weight/custom-test-yolo.cfg"
train_cfg = "/content/drive/MyDrive/Kobot/YOLO/darknet/cfg/yolov3.cfg"
#net = cv2.dnn.readNet("cfg", " weight")
net = cv2.dnn.readNetFromDarknet(test_cfg,Weights)

print(Weights, test_cfg)
classes = []
#with open("$path/classes.nemes" , "r") as f:
with open(path + "weight/classes.names" , "r") as f:
	classes = [line.strip() for line in f.readlines()]
print(classes)
color_lists = np.random.uniform(0, 255, size= (len(classes), 3))

layer_names = net.getLayerNames()
# print(layer_names)
output_layers = [layer_names[i[0] -1] for i in net.getUnconnectedOutLayers()]
# print(net.getUnconnectedOutLayers())
print(output_layers)

start_time = time.time()
img = cv2.imread(file_name)

h,w = img.shape[:2]
height = int(h * width / w)
print(height, width)

blob = cv2.dnn.blobFromImage(img, 0.00392, (416,416), swapRB=True, crop=False
							 )

net.setInput(blob)
outs = net.forward(output_layers)

confidences = []
names = []
boxes = []
colors = []

for out in outs:
	for detection in out:
		scores = detection[5:]
		class_id = np.argmax(scores)
		confidence = scores[class_id]
		if confidence > min_confidence:
			print(detection)
			# Object detected
			center_x = int(detection[0] * width)
			center_y = int(detection[1] * height)
			w = int(detection[2] * width)
			h = int(detection[3] * height)

			# Rectangle coordinates
			x = int(center_x - w /2)
			y = int(center_y - h / 2)

			boxes.append([x, y, w, h])
			confidences.append(float(confidence))
			names.append(classes[class_id])
			colors.append(color_lists[class_id])

indexes = cv2.dnn.NMSBoxes(boxes, confidences, min_confidence, 0.4)



font = cv2.FONT_HERSHEY_PLAIN
for i in range(len(boxes)):
	if i in indexes:
		x, y, w, h = boxes[i]
		label = str(names[i])
		con = (confidences[i] * 100)
		con = "{:.1f}".format(con)
		print (type(con))
		color = colors[i]
		#print(i, label, color, x, y, w, h)
		cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
		cv2.putText(img, con + "%", (x, y +80), font, 3, color, 3)
		cv2.putText(img, label, (x, y + 30), font, 3, color, 3)
  		


		

plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#cv2.imshow("Custom Yolo", file_name, img)
plt.show()
end_time = time.time()
process_time = end_time - start_time
print("===A frame took {:.3f} sec".format(process_time))
